In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import glob
import re


In [3]:
a = 10
b = 20
a + b


30

In [4]:
# aioushndiausndaiusndu

In [5]:
# Reza Alghif 

In [6]:
# test

In [7]:
# iqbal 2

In [8]:
# radiiiiiit

In [ ]:
# autosave kontolllll